In [14]:
!pip install folium

In [20]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

All Required Libraries Imported!


In [21]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))


Page download successful


In [18]:
import lxml
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

Postal Code           Borough               Neighborhood
0         M1A               NaN                        NaN
1         M2A               NaN                        NaN
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

In [23]:
df_html.dropna(subset=['Borough'], inplace=True)

In [25]:
df_html.describe()

Postal Code     Borough Neighborhood
count          103         103          103
unique         103          10           99
top            M4K  North York    Downsview
freq             1          24            4

In [27]:
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [28]:
df_html[df_html['Neighborhood'].isna()]

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []

In [30]:
df_html['Neighborhood'].fillna(df_html['Borough'], inplace=True)
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [49]:
df_html[df_html['Borough']=="Queen's Park"]

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []

In [56]:
df_postcodes =pd.DataFrame( df_html.groupby(['Postal Code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]))

In [57]:
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

Postal Code      Borough                            Neighborhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae

In [115]:
df_html.dropna(subset=['Borough'], inplace=True)

In [116]:
df_html.describe()

Postal Code     Borough Neighborhood
count          103         103          103
unique         103          10           99
top            M4K  North York    Downsview
freq             1          24            4

In [60]:
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [62]:
print('The shape of the dataset is:',df_postcodes.shape)

The shape of the dataset is: (103, 4)


In [63]:
df_postcodes.to_csv('Toronto_Postcodes.csv')

In [64]:
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [75]:
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

index Postal Code      Borough                            Neighborhood
0      0         M1B  Scarborough                          Malvern, Rouge
1      1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2      2         M1E  Scarborough       Guildwood, Morningside, West Hill
3      3         M1G  Scarborough                                  Woburn
4      4         M1H  Scarborough                               Cedarbrae

In [72]:
df_neighborhoods.describe()

index
count  103.000000
mean    51.000000
std     29.877528
min      0.000000
25%     25.500000
50%     51.000000
75%     76.500000
max    102.000000

In [73]:
df_coordinates.describe()

Latitude   Longitude
count  103.000000  103.000000
mean    43.704608  -79.397153
std      0.052463    0.097146
min     43.602414  -79.615819
25%     43.660567  -79.464763
50%     43.696948  -79.388790
75%     43.745320  -79.340923
max     43.836125  -79.160497

In [125]:
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postalcode': 'PostalCode'}, inplace=True)

In [126]:
df_neighborhoods.head()

index PostalCode      Borough                            Neighborhood
0      0        M1B  Scarborough                          Malvern, Rouge
1      1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2      2        M1E  Scarborough       Guildwood, Morningside, West Hill
3      3        M1G  Scarborough                                  Woburn
4      4        M1H  Scarborough                               Cedarbrae

In [127]:
df_neighborhoods.dropna(subset=['index'], inplace=True)

In [128]:
df_neighborhoods.head()

index PostalCode      Borough                            Neighborhood
0      0        M1B  Scarborough                          Malvern, Rouge
1      1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2      2        M1E  Scarborough       Guildwood, Morningside, West Hill
3      3        M1G  Scarborough                                  Woburn
4      4        M1H  Scarborough                               Cedarbrae

In [129]:
df_coordinates.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [130]:
df_neighborhoods.drop_duplicates()

index PostalCode           Borough  \
0        0        M1B       Scarborough   
1        1        M1C       Scarborough   
2        2        M1E       Scarborough   
3        3        M1G       Scarborough   
4        4        M1H       Scarborough   
5        5        M1J       Scarborough   
6        6        M1K       Scarborough   
7        7        M1L       Scarborough   
8        8        M1M       Scarborough   
9        9        M1N       Scarborough   
10      10        M1P       Scarborough   
11      11        M1R       Scarborough   
12      12        M1S       Scarborough   
13      13        M1T       Scarborough   
14      14        M1V       Scarborough   
15      15        M1W       Scarborough   
16      16        M1X       Scarborough   
17      17        M2H        North York   
18      18        M2J        North York   
19      19        M2K        North York   
20      20        M2L        North York   
21      21        M2M        North York   
22      22        M2N        North York   
23      23        M2P        North York   
24      24        M2R        North York   
25      25        M3A        North York   
26      26        M3B        North York   
27      27        M3C        North York   
28      28        M3H        North York   
29      29        M3J        North York   
30      30        M3K        North York   
31      31        M3L        North York   
32      32        M3M        North York   
33      33        M3N        North York   
34      34        M4A        North York   
35      35        M4B         East York   
36      36        M4C         East York   
37      37        M4E      East Toronto   
38      38        M4G         East York   
39      39        M4H         East York   
40      40        M4J         East York   
41      41        M4K      East Toronto   
42      42        M4L      East Toronto   
43      43        M4M      East Toronto   
44      44        M4N   Central Toronto   
45      45        M4P   Central Toronto   
46      46        M4R   Central Toronto   
47      47        M4S   Central Toronto   
48      48        M4T   Central Toronto   
49      49        M4V   Central Toronto   
50      50        M4W  Downtown Toronto   
51      51        M4X  Downtown Toronto   
52      52        M4Y  Downtown Toronto   
53      53        M5A  Downtown Toronto   
54      54        M5B  Downtown Toronto   
55      55        M5C  Downtown Toronto   
56      56        M5E  Downtown Toronto   
57      57        M5G  Downtown Toronto   
58      58        M5H  Downtown Toronto   
59      59        M5J  Downtown Toronto   
60      60        M5K  Downtown Toronto   
61      61        M5L  Downtown Toronto   
62      62        M5M        North York   
63      63        M5N   Central Toronto   
64      64        M5P   Central Toronto   
65      65        M5R   Central Toronto   
66      66        M5S  Downtown Toronto   
67      67        M5T  Downtown Toronto   
68      68        M5V  Downtown Toronto   
69      69        M5W  Downtown Toronto   
70      70        M5X  Downtown Toronto   
71      71        M6A        North York   
72      72        M6B        North York   
73      73        M6C              York   
74      74        M6E              York   
75      75        M6G  Downtown Toronto   
76      76        M6H      West Toronto   
77      77        M6J      West Toronto   
78      78        M6K      West Toronto   
79      79        M6L        North York   
80      80        M6M              York   
81      81        M6N              York   
82      82        M6P      West Toronto   
83      83        M6R      West Toronto   
84      84        M6S      West Toronto   
85      85        M7A  Downtown Toronto   
86      86        M7R       Mississauga   
87      87        M7Y      East Toronto   
88      88        M8V         Etobicoke   
89      89        M8W         Etobicoke   
90      90        M8X         Etobicoke   
91      91        M8Y         Etobicoke   
92    

In [131]:
df_coordinates.drop_duplicates()

PostalCode   Latitude  Longitude
0          M1B  43.806686 -79.194353
1          M1C  43.784535 -79.160497
2          M1E  43.763573 -79.188711
3          M1G  43.770992 -79.216917
4          M1H  43.773136 -79.239476
5          M1J  43.744734 -79.239476
6          M1K  43.727929 -79.262029
7          M1L  43.711112 -79.284577
8          M1M  43.716316 -79.239476
9          M1N  43.692657 -79.264848
10         M1P  43.757410 -79.273304
11         M1R  43.750072 -79.295849
12         M1S  43.794200 -79.262029
13         M1T  43.781638 -79.304302
14         M1V  43.815252 -79.284577
15         M1W  43.799525 -79.318389
16         M1X  43.836125 -79.205636
17         M2H  43.803762 -79.363452
18         M2J  43.778517 -79.346556
19         M2K  43.786947 -79.385975
20         M2L  43.757490 -79.374714
21         M2M  43.789053 -79.408493
22         M2N  43.770120 -79.408493
23         M2P  43.752758 -79.400049
24         M2R  43.782736 -79.442259
25         M3A  43.753259 -79.329656
26         M3B  43.745906 -79.352188
27         M3C  43.725900 -79.340923
28         M3H  43.754328 -79.442259
29         M3J  43.767980 -79.487262
30         M3K  43.737473 -79.464763
31         M3L  43.739015 -79.506944
32         M3M  43.728496 -79.495697
33         M3N  43.761631 -79.520999
34         M4A  43.725882 -79.315572
35         M4B  43.706397 -79.309937
36         M4C  43.695344 -79.318389
37         M4E  43.676357 -79.293031
38         M4G  43.709060 -79.363452
39         M4H  43.705369 -79.349372
40         M4J  43.685347 -79.338106
41         M4K  43.679557 -79.352188
42         M4L  43.668999 -79.315572
43         M4M  43.659526 -79.340923
44         M4N  43.728020 -79.388790
45         M4P  43.712751 -79.390197
46         M4R  43.715383 -79.405678
47         M4S  43.704324 -79.388790
48         M4T  43.689574 -79.383160
49         M4V  43.686412 -79.400049
50         M4W  43.679563 -79.377529
51         M4X  43.667967 -79.367675
52         M4Y  43.665860 -79.383160
53         M5A  43.654260 -79.360636
54         M5B  43.657162 -79.378937
55         M5C  43.651494 -79.375418
56         M5E  43.644771 -79.373306
57         M5G  43.657952 -79.387383
58         M5H  43.650571 -79.384568
59         M5J  43.640816 -79.381752
60         M5K  43.647177 -79.381576
61         M5L  43.648198 -79.379817
62         M5M  43.733283 -79.419750
63         M5N  43.711695 -79.416936
64         M5P  43.696948 -79.411307
65         M5R  43.672710 -79.405678
66         M5S  43.662696 -79.400049
67         M5T  43.653206 -79.400049
68         M5V  43.628947 -79.394420
69         M5W  43.646435 -79.374846
70         M5X  43.648429 -79.382280
71         M6A  43.718518 -79.464763
72         M6B  43.709577 -79.445073
73         M6C  43.693781 -79.428191
74         M6E  43.689026 -79.453512
75         M6G  43.669542 -79.422564
76         M6H  43.669005 -79.442259
77         M6J  43.647927 -79.419750
78         M6K  43.636847 -79.428191
79         M6L  43.713756 -79.490074
80         M6M  43.691116 -79.476013
81         M6N  43.673185 -79.487262
82         M6P  43.661608 -79.464763
83         M6R  43.648960 -79.456325
84         M6S  43.651571 -79.484450
85         M7A  43.662301 -79.389494
86         M7R  43.636966 -79.615819
87         M7Y  43.662744 -79.321558
88         M8V  43.605647 -79.501321
89         M8W  43.602414 -79.543484
90         M8X  43.653654 -79.506944
91         M8Y  43.636258 -79.498509
92         M8Z  43.628841 -79.520999
93         M9A  43.667856 -79.532242
94         M9B  43.650943 -79.554724
95         M9C  43.643515 -79.577201
96         M9L  43.756303 -79.565963
97         M9M  43.724766 -79.532242
98         M9N  43.706876 -79.518188
99         M9P  43.696319 -79.532242
100        M9R  43.688905 -79.554724
101        M9V  43.739416 -79.588437
102        M9W  43.706748 -79.594054

In [134]:
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')

In [135]:
# Check coordinates for a couple of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

index PostalCode           Borough        Neighborhood   Latitude  \
17     17        M2H        North York   Hillcrest Village  43.803762   
57     57        M5G  Downtown Toronto  Central Bay Street  43.657952   

    Longitude  
17 -79.363452  
57 -79.387383

In [136]:
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')

In [137]:
df = pd.read_csv('Toronto_Postcodes_2.csv', index_col=0)
df.head()

index PostalCode      Borough                            Neighborhood  \
0      0        M1B  Scarborough                          Malvern, Rouge   
1      1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek   
2      2        M1E  Scarborough       Guildwood, Morningside, West Hill   
3      3        M1G  Scarborough                                  Woburn   
4      4        M1H  Scarborough                               Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476

In [138]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)


The dataframe has 10 boroughs and 103 neighborhoods.


In [139]:
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [140]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

level_0 PostalCode          Borough                    Neighborhood  \
0       37        M4E     East Toronto                     The Beaches   
1       41        M4K     East Toronto    The Danforth West, Riverdale   
2       42        M4L     East Toronto  India Bazaar, The Beaches West   
3       43        M4M     East Toronto                 Studio District   
4       44        M4N  Central Toronto                   Lawrence Park   

    Latitude  Longitude  
0  43.676357 -79.293031  
1  43.679557 -79.352188  
2  43.668999 -79.315572  
3  43.659526 -79.340923  
4  43.728020 -79.388790

In [141]:
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [142]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()


In [143]:
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [144]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [150]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [155]:
CLIENT_ID = 'KIUEYR2NJV5NYKQTUHWGXHS2LUGUVF442FYPWQCEGLYDFGID'

CLIENT_SECRET = 'FLNQ3RR1FCOP02AUYZ3YQ0Q2VHI5SXQ3TYKGFNXKZJK1B10J'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [156]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [157]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [158]:
toronto_venues.shape

(1614, 7)

In [159]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            54   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing Centre, South Ce...                     19   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     63   
Christie                                                               17   
Church and Wellesley                                                   77   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             34   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           16   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
India Bazaar, The Beaches West                                         22   
Kensington Market, Chinatown, Grange Park                              60   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               45   
Moore Park, Summerhill East                                             4   
North Toronto West, Lawrence Park                                      22   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            34   
Regent Park, Harbourfront                                              44   
Richmond, Adelaide, King                                               94   
Rosedale                                                                4   
Roselawn                                                                4   
Runnymede, Swansea                                                     34   
St. James Town                                                         79   
St. James Town, Cabbagetown                                            43   
Stn A PO Boxes                                                         94   
Studio District                                                        40   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    21   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         34   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             54   
Brockton, Parkdale Village, Exhibition Place                            23   
Business reply mail Processing Centre, South Ce...                      19   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      63   
Christie                                                                17   
Church and Wellesley                                                    77   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              34   
Davisville North                    

In [160]:
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Restaurant',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Bubble Tea Shop',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Café', 'Indian Restaurant',
       'Japanese Restaurant', 'Lounge', 'Frozen Yogurt Shop',
       'American Restaurant', 'Liquor Store', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Park', 'Pet Store',
       'Steakhouse', 'Burrito Place', 'Movie Theater', 'Sandwich Place',
       'Light Rail Station', 'Fish Market', 'Gay Bar',
       'Seafood Restaurant', 'Cheese Shop', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Stationery Store', 'Wine Bar',
       'Thai Restaurant', 'Coworking Space', 'Latin American Restaurant',


In [161]:
"Indian Restaurant" in toronto_venues['Venue Category'].unique()

True

In [162]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1614, 237)


Neighborhoods  Afghan Restaurant  Airport  \
0                   The Beaches                  0        0   
1                   The Beaches                  0        0   
2                   The Beaches                  0        0   
3                   The Beaches                  0        0   
4  The Danforth West, Riverdale                  0        0   

   Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0                   0               0                0                 0   
1                   0               0                0                 0   
2                   0               0                0                 0   
3                   0               0                0                 0   
4                   0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Workshop  \
0                    0                 0                   0              0   
1                    0                 0                   0              0   
2                    0                 0                   0              0   
3                    0                 0                   0              0   
4                    0                 0                   0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  \
0          0           0           0       0     0    0                 0   
1          0           0           0       0     0    0                 0   
2          0           0           0       0     0    0                 0   
3          0           0           0       0     0    0                 0   
4          0           0           0       0     0    0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0       0              0          0         0   
1                   0       0              0          0         0   
2                   0       0              0          0         0   
3                   0       0              0          0         0   
4                   0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  Building  \
0                     0               0        0                0         0   
1                     0               0        0                0         0   
2                     0               0        0                0         0   
3                     0               0        0                0         0   
4                     0               0        0                0         0   

   Burger Joint  Burrito Place  Bus Line  Butcher  Café  \
0             0              0         0        0     0   
1             0              0         0        0     0   
2             0              0         0        0     0   
3             0              0         0        0     0   
4             0              0         0        0     0   

   Cajun / Creole Restaurant  Camera Store  Candy Store  Caribbean Restaurant  \
0                          0            

In [163]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 237)


Neighborhoods  Afghan Restaurant  \
0                                         Berczy Park           0.000000   
1        Brockton, Parkdale Village, Exhibition Place           0.000000   
2   Business reply mail Processing Centre, South C...           0.000000   
3   CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
4                                  Central Bay Street           0.000000   
5                                            Christie           0.000000   
6                                Church and Wellesley           0.012987   
7                      Commerce Court, Victoria Hotel           0.000000   
8                                          Davisville           0.000000   
9                                    Davisville North           0.000000   
10                       Dufferin, Dovercourt Village           0.000000   
11             First Canadian Place, Underground city           0.000000   
12    Forest Hill North & West, Forest Hill Road Park           0.000000   
13                           Garden District, Ryerson           0.000000   
14  Harbourfront East, Union Station, Toronto Islands           0.000000   
15                      High Park, The Junction South           0.000000   
16                     India Bazaar, The Beaches West           0.000000   
17          Kensington Market, Chinatown, Grange Park           0.000000   
18                                      Lawrence Park           0.000000   
19                           Little Portugal, Trinity           0.000000   
20                        Moore Park, Summerhill East           0.000000   
21                  North Toronto West, Lawrence Park           0.000000   
22                             Parkdale, Roncesvalles           0.000000   
23        Queen's Park, Ontario Provincial Government           0.000000   
24                          Regent Park, Harbourfront           0.000000   
25                           Richmond, Adelaide, King           0.000000   
26                                           Rosedale           0.000000   
27                                           Roselawn           0.000000   
28                                 Runnymede, Swansea           0.000000   
29                                     St. James Town           0.000000   
30                        St. James Town, Cabbagetown           0.000000   
31                                     Stn A PO Boxes           0.000000   
32                                    Studio District           0.000000   
33  Summerhill West, Rathnelly, South Hill, Forest...           0.000000   
34                The Annex, North Midtown, Yorkville           0.000000   
35                                        The Beaches           0.000000   
36                       The Danforth West, Riverdale           0.000000   
37           Toronto Dominion Centre, Design Exchange           0.000000   
38                     University of Toronto, Harbord           0.000000   

    Airport  Airport Food Court  Airport Lounge  Airport Service  \
0    0.0000              0.0000           0.000           0.0000   
1    0.0000              0.0000           0.000           0.0000   
2    0.0000              0.0000           0.000           0.0000   
3    0.0625              0.0625           0.125           0.1875   
4    0.0000              0.0000           0.000           0.0000   
5    0.0000              0.0000           0.000           0.0000   
6    0.0000              0.0000           0.000           0.0000   
7    0.0000              0.0000           0.000           0.0000   
8    0.0000              0.0000           0.000           0.0000   
9    0.0000              0.0000           0.000           0.0000   
10   0.0000              0.0000           0.000           0.0000   
11   0.0000              0.0000           0.000           0.0000   
12   0.0000              0.0000           0.000           0.0000   
13   0.0000              0.0000           0.000

In [164]:
len(to_grouped[to_grouped["Indian Restaurant"] > 0])

7

In [165]:
to_indian = to_grouped[["Neighborhoods","Indian Restaurant"]]

In [166]:
to_indian.head(9)

Neighborhoods  Indian Restaurant
0                                        Berczy Park           0.000000
1       Brockton, Parkdale Village, Exhibition Place           0.000000
2  Business reply mail Processing Centre, South C...           0.000000
3  CN Tower, King and Spadina, Railway Lands, Har...           0.000000
4                                 Central Bay Street           0.015873
5                                           Christie           0.000000
6                               Church and Wellesley           0.012987
7                     Commerce Court, Victoria Hotel           0.000000
8                                         Davisville           0.029412

In [167]:
from sklearn.cluster import KMeans
toclusters = 3

to_clustering = to_indian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [168]:
to_merged = to_indian.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

# Project Submission By -Niyati Sinha

In [169]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head(5)

Neighborhood  Indian Restaurant  \
0                                        Berczy Park           0.000000   
1       Brockton, Parkdale Village, Exhibition Place           0.000000   
2  Business reply mail Processing Centre, South C...           0.000000   
3  CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
4                                 Central Bay Street           0.015873   

   Cluster Labels  
0               0  
1               0  
2               0  
3               0  
4               1

In [170]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1614, 9)


Neighborhood  Indian Restaurant  Cluster Labels  Neighborhood Latitude  \
0  Berczy Park                0.0               0              43.644771   
0  Berczy Park                0.0               0              43.644771   
0  Berczy Park                0.0               0              43.644771   
0  Berczy Park                0.0               0              43.644771   
0  Berczy Park                0.0               0              43.644771   

   Neighborhood Longitude                                      Venue  \
0              -79.373306                                       LCBO   
0              -79.373306       The Keg Steakhouse + Bar - Esplanade   
0              -79.373306                             Fresh On Front   
0              -79.373306                              Meridian Hall   
0              -79.373306  Hockey Hall Of Fame (Hockey Hall of Fame)   

   Venue Latitude  Venue Longitude                 Venue Category  
0       43.642944       -79.372440                   Liquor Store  
0       43.646712       -79.374768                     Restaurant  
0       43.647815       -79.374453  Vegetarian / Vegan Restaurant  
0       43.646292       -79.376022                   Concert Hall  
0       43.646974       -79.377323                         Museum

In [171]:
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged.head()

Neighborhood  Indian Restaurant  Cluster Labels  \
0                Berczy Park                0.0               0   
25  Richmond, Adelaide, King                0.0               0   
25  Richmond, Adelaide, King                0.0               0   
25  Richmond, Adelaide, King                0.0               0   
25  Richmond, Adelaide, King                0.0               0   

    Neighborhood Latitude  Neighborhood Longitude  \
0               43.644771              -79.373306   
25              43.650571              -79.384568   
25              43.650571              -79.384568   
25              43.650571              -79.384568   
25              43.650571              -79.384568   

                                     Venue  Venue Latitude  Venue Longitude  \
0                                     LCBO       43.642944       -79.372440   
25                      Soho House Toronto       43.648734       -79.386541   
25                            Cafe Landwer       43.648753       -79.385367   
25  The Keg Steakhouse + Bar - York Street       43.649987       -79.384103   
25                               Rosalinda       43.650252       -79.385156   

                   Venue Category  
0                    Liquor Store  
25                      Speakeasy  
25                           Café  
25                     Restaurant  
25  Vegetarian / Vegan Restaurant

In [173]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Cluster Labels']):
    
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [180]:
#Cluster 0
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'Indian Restaurant') ]

Neighborhood  Indian Restaurant  Cluster Labels  \
36  The Danforth West, Riverdale           0.023256               1   
8                     Davisville           0.029412               1   
30   St. James Town, Cabbagetown           0.023256               1   
4             Central Bay Street           0.015873               1   
6           Church and Wellesley           0.012987               1   

    Neighborhood Latitude  Neighborhood Longitude                   Venue  \
36              43.679557              -79.352188           Sher-E-Punjab   
8               43.704324              -79.388790  Marigold Indian Bistro   
30              43.667967              -79.367675  Butter Chicken Factory   
4               43.657952              -79.387383         Colaba Junction   
6               43.665860              -79.383160   Kothur Indian Cuisine   

    Venue Latitude  Venue Longitude     Venue Category  
36       43.677308       -79.353066  Indian Restaurant  
8        43.702881       -79.388008  Indian Restaurant  
30       43.667072       -79.369184  Indian Restaurant  
4        43.660940       -79.385635  Indian Restaurant  
6        43.667872       -79.385659  Indian Restaurant

In [181]:
#Cluster 1
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Indian Restaurant') ]
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Indian Restaurant') ]

Neighborhood  Indian Restaurant  \
14  Harbourfront East, Union Station, Toronto Islands               0.01   

    Cluster Labels  Neighborhood Latitude  Neighborhood Longitude  \
14               0              43.640816              -79.381752   

                Venue  Venue Latitude  Venue Longitude     Venue Category  
14  Indian Roti House        43.63906       -79.385422  Indian Restaurant

In [182]:
#Cluster 2

to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['Venue Category'] == 'Indian Restaurant') ]

Neighborhood  Indian Restaurant  Cluster Labels  \
34  The Annex, North Midtown, Yorkville           0.047619               2   

    Neighborhood Latitude  Neighborhood Longitude                  Venue  \
34               43.67271              -79.405678  Roti Cuisine of India   

    Venue Latitude  Venue Longitude     Venue Category  
34       43.674618       -79.408249  Indian Restaurant

# Results

Most of the Indian restaurants are in cluster 2 with a value of Indian restaurants =0.047619

around coordinates:
    latitude:43.6
    longitudes:-79.3

And the minimum number of indian are in cluster 1 with a value of indian restaurants =0.01
Thus the model predicts the best place to open an indian restaurant is cluster 1